<a href="https://colab.research.google.com/github/budderpan/Personal_projects/blob/main/Chess%20bot/Chess_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NEXT STEP - indicates the next steps to be taken to develop the bot
#####ERROR - indicates a problem with the code

In [ ]:
import numpy as np

#creating the board
board = np.zeros((8,8))

#associating values with each piece
first_digit= {'white': '1','black':'2'}
first_digit_2 = {1:'white',2:'black'}
second_digit = {1:'pawn',2:'rook',3: "horse",4: 'bishop',5: 'queen',6:'king'}
piece_to_number = {'pawn':1,'rook':2,"horse":3, 'bishop':4, 'queen':5,'king':6}

#setting up the board
board[7] = list('23456432')
board[0] = list('23456432')
board[1] = list('1'*8)
board[6] = list('1'*8)

for a in range(len(board[7])):

  #making the pieces on one side white
  board[7][a] += 10
  board[6][a] += 10

  #making the pieces on the other side black
  board[0][a]+= 20
  board[1][a] += 20


original_board = board.copy()
print(board)



[[22. 23. 24. 25. 26. 24. 23. 22.]
 [21. 21. 21. 21. 21. 21. 21. 21.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [11. 11. 11. 11. 11. 11. 11. 11.]
 [12. 13. 14. 15. 16. 14. 13. 12.]]


##Functions defined for the code to work

In [ ]:
call_number = 0
def piece_ranges(first,second,possible_moves): #Used to find out the range of each piece

  #first: the amount by which the y value is displaced.
  #second: the amount by which the x value is displaced
    #Example: If first = 1 and second = 2, the function will check whether a new square that is 1 square below and 2 squares to the right of the selected square
    #         can be within the range of the selected piece or not

  #possible_moves: takes an empty list into which the possible moves are to be appended
  global increment
  global variable #variable = true when we are looking at opposition range
  global opposite_colour
  global selected_piece
  if piece_position[0]+first >= 0 and piece_position[1]+second >= 0: #this condition is necessary as (say) board[-1][7] can also give some valid position on the board
    if 0 == board[piece_position[0]+first][piece_position[1]+second] :
      possible_moves.append([piece_position[0]+first,piece_position[1]+second])

      increment += 1 #this causes the x and y value to increase accordingly, thus resulting in an increase in "first" and "second".
                 #In this way we are able to find out a series of squares along a single direction (like diagonal, left, right, etc.) and add them to the range of a piece

    elif variable == True:  #basically when variable == True (i.e king == True) then we are lookign at the opposition range.
                            #So this command says that if the current square of interest's value contains a particular piece (regardless of the colour),
                            #it will append that into the opponent_range list to make sure that the player's king
                            #does not move into the range of the opponent piece's after capturing a piece. that's why the opponent_range extends even to it's own pieces.

      possible_moves.append([piece_position[0]+first,piece_position[1]+second])
      raise IndexError


    elif str(board[piece_position[0]+first][piece_position[1]+second])[0] == opposite_colour and increment >= 0: #if opponent's piece in range
      possible_moves.append([piece_position[0]+first,piece_position[1]+second])
      raise IndexError

    else:
      raise IndexError
  else:
    raise IndexError



def shortened_piece_ranges(y,x,special): #Involves heavy reliance on piece_ranges function. It uses this function in order to obtain ranges of: rook, bishop, horse or king.
  global selected_piece
  global call_number
  global colour
  global king
  #y = amount by which y value should be incremented per new square while finding range
  #x = amount by which each x value should be incremented per new square while finding range. If x = 0 then that piece does not have a diagonal range
  #special = True only for king and horse.


  #notes: 1) direction of ranges (right, left, top-right, top left, etc.) are with respect to white being at the bottom of the board
  #       2)  top range means all the possible squares above a piece that the piece can move into. Same logic applied when referring to the other ranges

  #piece_range = 0: shortened_piece_ranges extracts top range when x = 0, top right range when x= 1, y = 1
  #piece_range = 1: shortened_piece_ranges extracts bottom range when x = 0, bottom left range when x= 1, y = 1
  #piece_range = 2: shortened_piece_ranges extracts right range when x = 0, bottom right range when x= 1, y = 1
  #piece_range = 3: shortened_piece_ranges extracts right range when x = 0, top left range when x= 1, y = 1

  piece_possible_moves = []
  piece_range = 0

  #call_number += 1
  #print(call_number, colour)
  while piece_range < 4:
    if special == False:
      global increment
      y = increment
      if x != 0:
        x = increment

    try:
      if piece_range == 0:
        piece_ranges(-y,x,piece_possible_moves)
      elif piece_range == 1:
        piece_ranges(y,-x,piece_possible_moves)
      elif piece_range == 2:
        piece_ranges(x,y,piece_possible_moves)
      elif piece_range == 3:
        piece_ranges(-x,-y,piece_possible_moves)
      if special == True: #if special = True then this is the case of all kings. thus piece_range will keep incrementing itself as kings can only move 1 square away
        piece_range+=1

    except IndexError:
      #print(piece_possible_moves, piece_range)
      piece_range +=1
      increment = 1

  return piece_possible_moves


def range_function(anticolour,selected_piece,king): #combines shortened_piece_ranges() to find entire range of all pieces except king

#anticolour: piece colour of the opponent.
#selected_piece: the piece that is selected/ the piece on the square selected by the player
#king: king takes a boolean value. king = True when selected piece is a king. This extra specification is made as king range does not exactly follow the norm for piece range.

  global variable
  global increment
  global opposite_colour
  global piece_position
  global move_history
  global en_passant
  global double_moved_pawn
  opposite_colour = anticolour
  variable = king

  #direction represents the direction along which the range should be taken. if the opponents colour is black then the players pieces are white.
  #thus if a pawn must move forward, its y value (in yx format) must reduce
  if anticolour == '2':
    direction = -1
  elif anticolour == '1':
    direction = 1

  #pawn
  if selected_piece[1] == '1':
    pawn_possible_moves = []

    #diagonal range of pawn (leftwards).
    try:
      if int(piece_position[0]+direction) >= 0 and int(piece_position[1])-1 >=0:
        if str(board[int(piece_position[0]+direction)][int(piece_position[1])-1])[0] == anticolour: #if there is an opponent piece diagonal (leftward) to pawn
          pawn_possible_moves.append([piece_position[0]+direction,piece_position[1]-1])
        elif king == True:
          pawn_possible_moves.append([piece_position[0]+direction,piece_position[1]-1])
        if move_history[-1] == [[int(piece_position[0])+direction*2,int(piece_position[1])-1],[int(piece_position[0]),int(piece_position[1])-1],anticolour + '1']:
          #en passant can only be possible if opponent's pawn moves two spaces exactly to the right or left of player's pawn and can only take place right after the opponent's pawn moves 2 spaces
          #thus the last played piece is the opponents pawn, which moves to and from defined positions with respect the the player's pawn
          #that is why this condition gets so specific

           pawn_possible_moves.append([piece_position[0]+direction,piece_position[1]-1])
           en_passant = [piece_position[0]+direction,piece_position[1]-1] #stores the en passant move to verify whether en passant takes place or not
           double_moved_pawn = [int(piece_position[0]),int(piece_position[1])-1] #used to remove the opponent's pawn if en passant is played
    except:
      pass

    #diagonal range of pawn (rightwards)
    try:
      if int(piece_position[0]+direction) >= 0 and int(piece_position[1])+1 >=0:
        if str(board[int(piece_position[0]+direction)][int(piece_position[1])+1])[0] == anticolour: #if there is an opponent piece diagonal (rightward) to pawn
          pawn_possible_moves.append([piece_position[0]+direction,piece_position[1]+1])
        elif king == True:
          pawn_possible_moves.append([piece_position[0]+direction,piece_position[1]+1])
        if move_history[-1] == [[int(piece_position[0])+direction*2,int(piece_position[1])+1],[int(piece_position[0]),int(piece_position[1])+1],anticolour + '1']:
           pawn_possible_moves.append([piece_position[0]+direction,piece_position[1]+1])
           en_passant = [piece_position[0]+direction,piece_position[1]+1]
           double_moved_pawn = [int(piece_position[0]),int(piece_position[1])+1]

    except:
      pass

    #forward movement of pawns
    if king != True: #a king can move to a square infront of a pawn even if it is int he pawn's range, thus forward movement is not considered when king == True

      #one square forward
      try:
        if 0 == board[int(piece_position[0]+direction)][int(piece_position[1])] and int(piece_position[0]+direction) >= 0:
          pawn_possible_moves.append([piece_position[0]+direction,piece_position[1]])

          #two squares forward special move
          #checks whether the selected pawn has been moved before
          #ERROR (Optimization-Related): Code can be made faster by checking whether a black pawn is in column 1 and whether a white pawn is in column 6
          for a in move_history:
            if piece_position in a:
              raise ZeroDivisionError

          if 0 == board[int(piece_position[0]+(2*direction))][int(piece_position[1])]:
            pawn_possible_moves.append([piece_position[0]+(2*direction),piece_position[1]])
      except:
        pass

    return pawn_possible_moves

  #rook
  elif selected_piece[1] == '2':
    increment = 1
    return shortened_piece_ranges(increment,0,False)

  #bishop
  elif selected_piece[1] == '4':
    increment = 1
    return shortened_piece_ranges(increment,increment,False)

  #queen
  elif selected_piece[1] == '5':
    increment = 1
    return shortened_piece_ranges(increment,0,False) + shortened_piece_ranges(increment,increment,False)

  #horse
  elif selected_piece[1] == '3':
    return shortened_piece_ranges(2,1,True) + shortened_piece_ranges(1,2,True)

  #king
  elif selected_piece[1] == '6' and king == True:
    return shortened_piece_ranges(1,1,True) + shortened_piece_ranges(1,0,True)


def piece_range_display(colour): #displays the entire range of a certain piece including king with the help of range_function()
  #piece_range_display(colour)[0] shows the visual representation of the range of the selected piece
  #piece_range_display(colour)[1] consists of the coordinates to which the piece can move

  global selected_coin
  global board
  global original_piece_position
  global piece_position
  global opposition_range
  global original_board
  global move_history

  returnable = [] #piece_range_display returns this variable as the output
  piece_range_coordinates = [] #used to store coordinates of all the squares within the range of the selected piece

  if colour == '2':
    other_colour = '1'
  elif colour == '1':
    other_colour = '2'

  if selected_coin[1] != '6':
    j = range_function(other_colour,selected_coin,False)
    for a in j:
      board[a[0]][a[1]] = 99 #creating visual representation of range of piece
      piece_range_coordinates.append([a[0],a[1]])
    returnable = [board,piece_range_coordinates]
    board = original_board.copy() #over here original_board is the board before any move is made
                                  #(since this function is called before original_board changes to the new board setup created once a move is made)
    return returnable

  elif selected_coin[1] == '6':
    opposition_range = [] #coordinates of the ranges of all the opponent pieces combined
    board[original_piece_position[0]][original_piece_position[1]] = 0 #this is necessary in order to get the entire opponent range.
    # Example: a king is in the way of a rook, it cannot move back because it will still be in the range of the rook upon moving back
    # if the king was not removed from the board temporarily, the range of the rook would end at the king's position, thus allowing it to move back. But this is not the case

    #gets the range of all the opponent and stores them in opposition_range

    for a in range(len(board)):
      for b in range(len(board[a])):
        if str(int(board[a][b]))[0] == other_colour:
          piece_position = [a,b]
          opposition_range += range_function(colour,str(int(board[a][b])),True)

          #print(range_function('2',str(int(board[a][b])),True))
          #print(first_digit_2[int(str(int(board[a][b]))[0])],second_digit[int(str(int(board[a][b]))[1])])
    '''iteration_number = -1
    for piece_category in piece_range_list[opp]:
      for specific_piece in piece_range_list[opp][piece_category]:
        iteration_number += 1
        if piece_category == "pawn":
          for each_move in piece_range_list[opp]["pawn"][iteration_number]:
            if piece_position_list[opp]["pawn"][iteration_number][1] != piece_range_list[opp]["pawn"][iteration_number][each_move][1]:
              opposition_range.append(piece_range_list[opp]["pawn"][iteration_number][each_move])
        else:
          opposition_range += specific_piece'''



    piece_position = original_piece_position.copy()
    board[original_piece_position[0]][original_piece_position[1]] = selected_coin #putting the king back in its place
    king_range = shortened_piece_ranges(1,1,True) + shortened_piece_ranges(1,0,True) #range of player's king
    #print(opposition_range)
    #print(king_range)
    king_range = list(filter(lambda x: x not in opposition_range,king_range)) #makes sure king cannot move in illegal squares
    king_range = list(filter(lambda x: str(int(board[x[0]][x[1]]))[0] != colour,king_range)) #makes sure king cannot move into squares occupied by white pieces

    #criteria for castling
    try:
      rook_1 = ''
      rook_2 = ''
      for a in move_history:
        if original_piece_position in a: #if king has been moved
          raise ZeroDivisionError
        if colour == '1':
          if [7,0] in a:
            rook_1 = 'nope'
          if [7,7] in a:
            rook_2 = 'nope'
        else:
          if [0,0] in a :
            rook_1 = 'nope'
          if [0,7] in a:
            rook_2 = 'nope'

      #left side castling criteria (with respect to white at the bottom of the board)
      if rook_1 == '':
        if original_piece_position not in opposition_range and [original_piece_position[0],original_piece_position[1]-2] not in opposition_range and [original_piece_position[0],original_piece_position[1]-1] in king_range and int(board[original_piece_position[0]][original_piece_position[1]-1]) == 0 and int(board[original_piece_position[0]][original_piece_position[1]-3]) == 0 and int(board[original_piece_position[0]][original_piece_position[1]-2])==0:
          king_range.append([original_piece_position[0],original_piece_position[1]-2])

      #right side castling criteria (with respect to white at the bottom of the board)
      if rook_2 == '':
        if original_piece_position not in opposition_range and [original_piece_position[0],original_piece_position[1]+2] not in opposition_range and int(board[original_piece_position[0]][original_piece_position[1]+2])==0 and int(board[original_piece_position[0]][original_piece_position[1]+1])==0 and [original_piece_position[0],original_piece_position[1]+1] in king_range:
          king_range.append([original_piece_position[0],original_piece_position[1]+2])

    except ZeroDivisionError:
      pass



    for d in king_range:
      try:
        if d[0] >= 0 and d[1] >=0:
          board[d[0]][d[1]] = 99
          piece_range_coordinates.append([d[0],d[1]])
      except:
        pass
    returnable = [board,piece_range_coordinates]
    board = original_board.copy()
    return returnable

def full_range(colour_of_side): #colour_of_side is '1' (white) or '2' (black)

  range_of_side = []
  global original_values
  global selected_coin
  global board
  global original_piece_position
  global piece_position
  global opposition_range
  global original_board
  global move_history

  for a in range(len(board)):
    for b in range(len(board[a])):
      if str(int(board[a][b]))[0] == colour_of_side: #if the for loop encounters any piece from the specified colour

        #changing the values according to the piece encountered by the for loop
        selected_coin = str(int(board[a][b]))
        piece_position = [a,b]
        original_piece_position = piece_position.copy()
        range_of_side += piece_range_display(colour_of_side)[1] #appending the range of each piece into the range_of_side list

  #converting the variables back to their original values
  piece_position = original_values['piece_position']
  original_piece_position = original_values['original_piece_position']
  selected_coin = original_values['selected_coin']
  colour = original_values['colour']

  return range_of_side

def input_text_filter(text): #to accept even horribly spaced/capitalized inputs
  return "".join(text.split()).strip().lower()



ValueError: list.remove(x): x not in list

In [ ]:
board = np.zeros((8,8))

board[0] = np.array([0, 0, 26, 22, 0, 0, 0, 22])
board[1] = np.array([21, 0, 11, 0, 0, 15, 21, 21])
board[2] = np.array([0, 0, 0, 0, 0, 0, 24, 0])
board[3] = np.array([0, 14, 11, 21, 23, 0, 0, 0])
board[4] = np.array([0, 15, 0, 0,11, 0, 0, 0])
board[5] = np.array([11, 0, 14, 0, 0, 0, 0, 0])
board[6] = np.array([14, 11, 0, 0, 0, 21, 0, 11])
board[7] = np.array([12, 0, 0, 0, 16, 0, 0, 12])

original_board = board.copy()
print(board)

[[ 0.  0. 26. 22.  0.  0.  0. 22.]
 [21.  0. 11.  0.  0. 15. 21. 21.]
 [ 0.  0.  0.  0.  0.  0. 24.  0.]
 [ 0. 14. 11. 21. 23.  0.  0.  0.]
 [ 0. 15.  0.  0. 11.  0.  0.  0.]
 [11.  0. 14.  0.  0.  0.  0.  0.]
 [14. 11.  0.  0.  0. 21.  0. 11.]
 [12.  0.  0.  0. 16.  0.  0. 12.]]


In [ ]:
print(piece_range_list[0]["king"])

[[[6, 6], [7, 6], [7, 5]]]


###Code for training the model

In [ ]:
position_conversion = {"a":0,"b":1,"c":2,"d":3,"e":4,"f":5,"g":6,"h":7} #for converting position letters in the notation to according index numbers
black_piece_positions = {"pawn":[],"rook":[],"horse":[],"bishop":[],"queen":[],"king":[]} #should store the positions of all the pieces
black_piece_ranges = {"pawn":[],"rook":[],"horse":[],"bishop":[],"queen":[],"king":[]} #should store the range of each piece for all pieces
white_piece_positions = {"pawn":[],"rook":[],"horse":[],"bishop":[],"queen":[],"king":[]} #should store the positions of all the pieces
white_piece_ranges = {"pawn":[],"rook":[],"horse":[],"bishop":[],"queen":[],"king":[]} #should store the range of each piece for all pieces

first_digit= {'white': '1','black':'2'}
first_digit_2 = {1:'white',2:'black'}
second_digit = {1:'pawn',2:'rook',3: "horse",4: 'bishop',5: 'queen',6:'king'}
piece_to_number = {'pawn':1,'rook':2,"horse":3, 'bishop':4, 'queen':5,'king':6}
notation_to_piece = {"R":"rook","N": "horse", "B": "bishop","Q": "queen","K":"king"}

for a in range(len(board)):
  for b in range(len(board[a])):
    selected_coin = str(int(board[a][b]))
    if selected_coin != "0":
      piece_position = [a,b]
      original_piece_position = piece_position
      move_history = []
      if str(selected_coin[0]) == "1":
        white_piece_ranges[second_digit[int(selected_coin[1])]] += [piece_range_display("1")[1]]
        white_piece_positions[second_digit[int(selected_coin[1])]] += [[a,b]]
      else:
        black_piece_ranges[second_digit[int(selected_coin[1])]] += [piece_range_display("2")[1]]
        black_piece_positions[second_digit[int(selected_coin[1])]] += [[a,b]]
piece_position_list = [white_piece_positions,black_piece_positions] #list[dicts{strs:list[lists]}]    Example: piece_position_list[0]["king"] == [[7,4]]
piece_range_list = [white_piece_ranges,black_piece_ranges]  #list[dicts{strs:list[lists[lists]]}]     Example: piece_range_list[0]["king"] = [[[7, 5], [7, 3], [7, 2]]]

In [ ]:
#NEXT STEPS: Check whether a certain move is possible or not by checking if it is in piece_range_display(colour)[1] or not.
            #Do this once moving_piece, piece_position, selected_coin and colour are defined
#NEXT STEPS: Be able to print out an error and continue parsing through chess training data by skipping that iteration using continue function
#ERROR: En passant and castling has not been tested for when move_history != []


#Code for Chess notation conversion

#thigs that are used:
#piece_position (the position of the selected piece)
#selected_coin (the piece selected example: black horse = 13)
#colour (colour of the selected piece)
#moving_piece (position to which the piece moves)

check = False
draw = False
checkmate = False

#Original values

original_values = {}

castle_side = 0
turn = 1
if turn ==1:
  opp = 0
else:
  opp = 1

move_history = [] #NEXT STEP: append moves to move_history

#Setting the new positions
def make_move(): #TESTED
  global moving_piece
  global board
  global piece_position
  global selected_coin
  global original_board
  global move_history
  board[int(moving_piece[0])][int(moving_piece[1])] = selected_coin             #the selected piece is now moved to the new position
  board[piece_position[0]][piece_position[1]] = 0            #old position of piece is vacated (by turning the value of the position to 0)
  original_board = board.copy()
  move_history.append([piece_position,list(map(int,list(moving_piece))),selected_coin])

def get_new_range(sc,pp,piece,piece_number): #TESTED

  #sc - str. Example: "12" (white rook)
  #pp - list. Example: [1,2] ([y,x] format) - position of the selected piece
  #piece - str. Example: "rook"
  #piece_number - int - index at which the position/range of the particular piece is stored in piece_positions and piece_ranges dictionaries
  global selected_coin
  global piece_position_list
  global piece_range_list
  global piece_position
  global original_board
  global board
  global turn
  global move_history
  selected_coin = sc
  piece_position = pp
  original_piece_position = piece_position.copy()
  piece_position_list[turn][piece][piece_number] = piece_position.copy()
  piece_range_list[turn][piece][piece_number] = piece_range_display(str(turn+1))[1].copy()



#removing the taken piece
def remove_piece(taken_piece): #PARTIALLY TESTED
  #taken_piece: position of the piece that is captured - [y,x] format
  global piece_position_list
  global piece_range_list
  global turn
  global opp
  global second_digit


  for a in range(len(piece_position_list[opp])):
    breaker = 0
    for b in range(len(piece_position_list[opp][second_digit[a+1]])):


      if piece_position_list[opp][second_digit[a+1]][b] == taken_piece:
        piece_position_list[opp][second_digit[a+1]][b] = []
        piece_range_list[opp][second_digit[a+1]][b] = []
        breaker = 1
        break
    if breaker == 1:
        print("remove the taken piece from piece_positions and piece_ranges dictionaries")
        break

#while statement to be added here

#checking for a draw/checkmate/check. Move this entire block of code to the last part of the code
check_counter = 0
horse_check = []
iteration_number = -1
check = False
draw = False
stalemate = False
for piece_category in piece_range_list[opp]:
  breaker = 0
  for specific_piece in piece_range_list[opp][piece_category]:
    iteration_number += 1
    if piece_position_list[turn]["king"][0] in specific_piece:
      check = True
      if piece_category == "horse":
        horse_check = piece_position_list[opp]["horse"][iteration_number]
      check_counter += 1
      breaker = 1
      break
  if breaker == 1:
    break
for piece_category in piece_range_list[turn]:
  breaker = 0
  for specific_piece in piece_range_list[turn][piece_category]:
    if specific_piece != []:
      breaker = 1
      break
  if breaker == 1:
    break
  if piece_category == "king":
    draw = True
    stalemate = True
if check:
  allowed_positions = []
  if check_counter == 1 and horse_check == []:
    board[piece_position_list[turn]["king"][0][0]][piece_position_list[turn]["king"][0][1]] = int(str(turn + 1) + "5")
    piece_position = piece_position_list[turn]["king"][0]
    original_piece_position = piece_position.copy()
    selected_coin = int(str(turn + 1) + "5")
    check_counter = 0
    allowed_positions = []

    for attacks in piece_range_display(colour)[1]:
      for a in piece_position_list[opp]:
        if a in ["rook","bishop","queen"]:
          if attacks in piece_position_list[opp][a]:
            allowed_positions += [attacks]
            allowed_positions += piece_range_list[opp][a][piece_position_list[opp][a].index(attacks)].intersection(piece_range_display(colour)[1])

  elif horse_check != []:
    allowed_positions += [horse_check]
  if stalemate == True:
    print("Checkmate!")
  if piece_range_list[turn]["king"][0] == []:
    for piece_category in ["pawn","rook","horse","bishop","queen"]:
      breaker = 0
      for specific_piece in piece_range_list[turn][piece_category]:
        if specific_piece.intersection(allowed_positions) != []:
          breaker = 1
          break
      if breaker == 1:
        break
      if piece_category == "queen":
        print("Checkmate!")

#NEXT STEP: while statement to be added here such that everytime an invalid move is made it is redirected back here
notation = input("Make your move using standard chess notation: ")

if check_counter >1:
  if notation[0] != "K":
    print("Invalid move. King is under check!")

#for en passant
if turn == 1:
  colour = first_digit["black"]
  opposite_increment = -1
  opp = 0
  starting_row_number = 0
else:
  colour = first_digit["white"]
  opposite_increment = 1
  opp = 1
  starting_row_number = 7


#checking for castling notations
if "O-O-O" in notation or "0-0-0" in notation: #TESTED

  #checking for whether castling is possible or not
  #ERROR: King Cannot move into a square which is within the opponent range. Test whether castling is possible in such a case
  if [piece_position_list[turn]["king"][0][0],[piece_position_list[turn]["king"][0][1]-2]] not in piece_range_list[turn]["king"][0]:
    print("Long castling is not possible")


  else:
    piece_position = piece_position_list[turn]["king"][0]
    castle_side = -2 #long hand castling
    moving_piece = [piece_position[0],piece_position[1] - 2]
    selected_coin = colour + '6'
    board[piece_position[0]][piece_position[1]-1] = board[piece_position[0]][piece_position[1]-4]
    board[piece_position[0]][piece_position[1]-4] = 0

    make_move()
    get_new_range(colour+"2",[piece_position[0],piece_position[1]-1],"rook",0)

    #ERROR: get_new_range(colour+"6",moving_piece,"king",0) can be used to replace this block of code, but it is not working for some reason
    selected_coin = colour +"6"
    piece_position = moving_piece
    piece = "king"
    piece_number = 0
    original_piece_position = piece_position.copy()
    piece_position_list[turn][piece][piece_number] = piece_position.copy()
    piece_range_list[turn][piece][piece_number] = piece_range_display(str(turn+1))[1].copy()

    print("long hand castling")
  #continue

elif "O-O" in notation or "0-0" in notation: #TESTED (EXCEPT THE ERROR)

  #checking for whether castling is possible or not
  if [piece_position_list[turn]["king"][0][0],[piece_position_list[turn]["king"][0][1]+2]] not in piece_range_list[turn]["king"][0]:
    print("Long castling is not possible")

  else:
    castle_side = 2 #short hand castling
    selected_coin = colour + '6'
    piece_position = piece_position_list[turn]["king"][0]
    moving_piece = [piece_position[0],piece_position[1] + 2]
    board[piece_position[0]][piece_position[1]+1] = colour + "2" #ERROR: King range is wrong when this position is set to 0
    board[piece_position[0]][piece_position[1]+3] = 0
    print(moving_piece)
    make_move()

    get_new_range(colour + "2",[piece_position[0],piece_position[1]+1],"rook",1)

    #ERROR: get_new_range(colour+"6",moving_piece,"king",0) can be used to replace this block of code, but it is not working for some reason
    selected_coin = colour +"6"
    piece_position = moving_piece
    piece = "king"
    piece_number = 0
    original_piece_position = piece_position.copy()
    piece_position_list[turn][piece][piece_number] = piece_position.copy()
    piece_range_list[turn][piece][piece_number] = piece_range_display(str(turn+1))[1].copy()

    print("short hand castling")
  #continue

#to directly identify check while training
if notation[-1] == '+':
  #potential_check = True. veriication of check can be done at the end of the code
  check = True #this should affect the next iteration. i.e it affects the opponent's turn
  notation = notation[0:-1]
  print("Potential Check!")


#to directly identify checkmate while training
elif notation[-1] == "#":
  checkmate = True #potential_checkmate = True. veriication of checkmate can be done at the end of the code
  notation = notation[0:-1]
  print("Potential Checkmate!")
  #break
# else:
#  check = False

#if a piece is captured then the position to which it moves is defined after the x in the notation
if "x" in notation: #PARTIALLY TESTED
  moving_piece = notation[notation.index("x")+1:notation.index("x")+3]
  moving_piece = str(position_conversion[moving_piece[0]]) +str(8 - int(moving_piece[1])) #converts standard notation to bot readable form
                                                          #we do str(int(moving_piece[1])-1 as our column index goes 0,1,2,3,4 but in standard notation it goes 1,2,3,4,5
  moving_piece = moving_piece[::-1]

  original_values["moving_piece"] = moving_piece

#pawn notation conversion
if notation[0] not in ['R','N','B','Q','K']:
  selected_coin = colour +'1'
  original_values["selected_coin"] = selected_coin

  if "x" in notation:
    taken_piece_position = []
    #en passant (en passant occurs only if there is x in the notation but the pawn moves to an empty square. In such a case the pawn below the moving pawn is captured)
    if int(board[int(moving_piece[0])][int(moving_piece[1])]) == 0:

      # position of the opponent's pawn being taken [y,x] format
      taken_piece_position = [int(moving_piece[0])+ opposite_increment, int(moving_piece[1])]
      print("taken_piece_position =", taken_piece_position)
      board[taken_piece_position[0]][taken_piece_position[1]] = 0


    if taken_piece_position == []:
      taken_piece_position = [int(moving_piece[0]),int(moving_piece[1])]

    remove_piece(taken_piece_position)

    #checking which pawn's position matches the letter used before "x" to describe the pawn that captures a piece
    for a in range(len(piece_position_list[turn]["pawn"])): #TESTED
      if position_conversion[notation[0]] == piece_position_list[turn]["pawn"][a][1] and list(map(int,list(moving_piece))) in piece_range_list[turn]["pawn"][a]:
        piece_position = piece_position_list[turn]["pawn"][a]
        original_values["piece_position"] = piece_position.copy()
        make_move()
        get_new_range(selected_coin,[int(moving_piece[0]),int(moving_piece[1])],"pawn",a)

        print("get new pawn range and position and remove old pawn. Make changes accordingly in piece_positions and piece_ranges dictionaries ")
        break






  #standard pawn notation case
  else:
    moving_piece = notation[0:2]
    moving_piece = str(position_conversion[moving_piece[0]]) +str(8 - int(moving_piece[1])) #converts standard notation to bot readable form
    moving_piece = moving_piece[::-1]
    original_values["moving_piece"] = moving_piece


    #checking which pawn can move to the position mentioned in the standard notation
    for a in range(len(piece_position_list[turn]["pawn"])):
      if list(map(int,list(moving_piece))) in piece_range_list[turn]["pawn"][a]:

        piece_position = piece_position_list[turn]["pawn"][a]

        make_move()
        get_new_range(selected_coin,[int(moving_piece[0]),int(moving_piece[1])],"pawn",a)

        print("Get new pawn range and position")
        break



  #for pawn promotion
  if "=" in notation: #TESTED
    promotion = notation_to_piece[notation[notation.index("=")+1]]
    selected_coin = colour + str(piece_to_number[promotion])
    board[int(moving_piece[0])][int(moving_piece[1])] = selected_coin

    #removing the pawn from the board (this code is faster than remove_piece() so remove_piece() is not used)
    piece_position_list[turn]["pawn"][a] = []
    piece_range_list[turn]["pawn"][a] = []
    print("promotion = ", promotion)

    #setting up range and position of new piece
    piece_position_list[turn][promotion] += [[]]
    piece_range_list[turn][promotion] += [[]]
    get_new_range(selected_coin, [int(moving_piece[0]),int(moving_piece[1])],promotion,-1)

    print("pawn promoted to",promotion)
    print("add new piece to piece_positions and piece_ranges dictionaries")





#notation conversion for all other pieces
elif notation[0] in ['R','N','B','Q','K']: #TESTED PARTIALLY (Untested for wrong inputs)
                                          #ERROR: If two same pieces can move to the same location, but wrong input is given, then the first bishop that can move to that location is moved.
                                                  #FIX: Check whether two pieces can move to a particular location. If so mark it as wrong input.
  selected_coin = colour + str(piece_to_number[notation_to_piece[notation[0]]]) #1st letter of notation indicates the piece
  notation = notation[1:]
  if "x" in notation:
    notation = notation[0:notation.index("x")] + notation[notation.index("x")+1:] #x does not make a difference whether it exists or not in the notation.
                                        #The bot works in such a way that if a piece moves to a square containing another piece, the other piece is automatically captured
    print("remove the taken piece from piece_ranges and piece_positions dictionary")
    remove_piece([int(moving_piece[0]),int(moving_piece[1])])

  moving_piece = notation[-2:]
  moving_piece = str(position_conversion[moving_piece[0]]) +str(8 - int(moving_piece[1]))
  moving_piece = moving_piece[::-1]
  notation = notation[0:-2]

  piece_position = []


  #if there are two of the same pieces that can move to the same square
  if len(notation) == 1:
    for a in range(len(piece_position_list[turn][second_digit[int(selected_coin[1])]])):
      try:
        if int(notation) == piece_position_list[turn][second_digit[int(selected_coin[1])]][a][0]:
          piece_position = piece_position_list[turn][second_digit[int(selected_coin[1])]][a]
          break
      except ValueError:
        if position_conversion[notation] == piece_position_list[turn][second_digit[int(selected_coin[1])]][a][1]:
          piece_position = piece_position_list[turn][second_digit[int(selected_coin[1])]][a]
          break




  #if there are three of the same pieces that can move to the same square such that two of them are located in the same row and the other two are located in the same column (very rare)
  elif len(notation) == 2:
    piece_position = [8 - int(notation[1]),position_conversion[notation[0]]]


  else:
    piece = second_digit[int(selected_coin[1])]
    for a in range(len(piece_position_list[turn][piece])):
      if list(map(int,list(moving_piece))) in piece_range_list[turn][piece][a]:
        piece_position = piece_position_list[turn][piece][a]
        break
  if piece_position == []:
    print("piece cannot move there")
  else:
    make_move()
    print(piece_position)
    for a in range(len(piece_position_list[turn][second_digit[int(selected_coin[1])]])):
      if piece_position == piece_position_list[turn][second_digit[int(selected_coin[1])]][a]:
        get_new_range(selected_coin,list(map(int,list(moving_piece))),second_digit[int(selected_coin[1])],a)
        break
    print("updated piece_positions and piece_ranges dictionaries accordingly")
print("moving_piece = ", moving_piece)
print("colour", colour)
print("selected_coin", selected_coin)
print("piece_position = ", piece_position)
print(board)


Make your move using standard chess notation: 0-0
Long castling is not possible


KeyError: '0'

In [ ]:
print(piece_range_list[0]["king"])
print(piece_position_list[0]["king"])

[[[7, 5], [7, 3], [7, 2]]]
[[7, 4]]


In [ ]:
board = original_board.copy()
for a in piece_range_list[0]["king"][0]:
  board[a[0]][a[1]] = 99
board


array([[ 0.,  0., 26., 22.,  0.,  0.,  0., 22.],
       [21.,  0., 11.,  0.,  0., 15., 21., 21.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 24.,  0.],
       [ 0., 14., 11., 21., 23.,  0.,  0.,  0.],
       [ 0., 15.,  0.,  0., 11.,  0.,  0.,  0.],
       [11.,  0., 14.,  0.,  0.,  0.,  0.,  0.],
       [14., 11.,  0.,  0.,  0., 11., 99., 11.],
       [12.,  0.,  0.,  0., 99., 99., 16., 99.]])

##Beginning of the code

In [ ]:
#NEXT STEP: Convert standard chess notations to bot readable notations
#NEXT STEP: Use Piece_ranges and piece_positions dictionaries to store and update the positions and ranges of each piece
#ERROR: Nothing new happens when a player is under check
#user chooses between black or white.
#While loop exists to make sure user chooses only black or white and not any other option
while True:
  side = input_text_filter(input('Choose your side : 1) Black 2) White'))
  if side in ['black','white']:
    break
  else:
    print('Pick a valid option!')


move_history = []

#player's side is stored in turn
turn = side

#if any of these variables become 'True', the game ends
checkmate = False
draw= False

#working of the game
while checkmate == False and draw == False:
  global opposition_range
  opposition_range = []
  while True:
    #variables for en passant
    en_passant = []
    double_moved_pawn = []
    try:
      print(original_board)
      selected_square = list(input("Choose a piece's square in the format 'yx' or resign \n y = y-coordinate\n x = x-coordinate"))

      #if player types resign then game ends
      if "".join(selected_square).strip().lower()== 'resign':
        if turn == "black":
          print('White wins!')
        elif turn == "white":
          print("Black wins!")
        checkmate = True
        break

      #if the selected square does not contain any piece (==0) or contains a piece that is not the player's i.e. opponent's piece then error is thrown which is handled accordingly
      elif board[int(selected_square[0])][int(selected_square[1])] == 0 or first_digit_2[int(str(board[int(selected_square[0])][int(selected_square[1])])[0])] != turn:
        print("The piece you selected is either the opponent's or there is no piece on the selected square")
        raise ZeroDivisionError

      else:
        piece_position = list(map(int,selected_square))       #piece_position = position of the piece selected by the player
        break
    except KeyboardInterrupt:
      raise KeyboardInterrupt

    #handles any errors thrown by invalid values for 'selected_square' variable
    except:
      print('Choose a valid piece position\n')
      pass


  #ends the game
  if checkmate == True:
    break

  original_piece_position = piece_position[:] #initial position of the selected piece
  selected_coin = str(int(board[int(piece_position[0])][int(piece_position[1])])) #selected coin = piece that is selected
  colour = selected_coin[0]

  #stores all info about a certain selected piece
  original_values = {'piece_position': piece_position.copy(),
                'original_piece_position':original_piece_position.copy(),
                'selected_coin': selected_coin,
                'colour': colour}
  if piece_range_display(colour)[1] == []:
    print("\n This piece cannot move anywhere. Choose a valid piece position \n")
    continue
  #NEXT STEPS: FIND MORE CRITERIA FOR DRAWS IN A CHESS GAME. ONLY STALEMATE HAS BEEN COVERED. INCLUDE DRAW BY REPETITION AND LACK OF MATERIAL
  try:
    #checking for a draw/checkmate/check.
    check = False
    draw = True
    if colour == '1':
      other_colour = '2'
    else:
      other_colour = '1'
    opposition_range = full_range(other_colour)
    print(original_values)
    for a in range(len(board)):
      for b in range(len(board[a])):
        if str(board[a][b])[0] == first_digit[turn]: #if the for loops encounter any of the player's piece

          if int(str(int(board[a][b]))[1]) == 6: #if the player's piece encountered by the for loops is a king

            #if there are no possible moves that can be played by the player and their king is under check, then the player is checkmated
            if full_range(colour) == [] and [a,b] in opposition_range:


              print('Checkmate. Computer Wins!')
              checkmate = True
              raise SyntaxError

            elif [a,b] in opposition_range:   #if king is under check
              print('Check!')
              check = True

        if full_range(colour) != []:   #if the player cannot make a move then its a draw
          draw = False
    if draw == True:
      print('Draw!')
      raise SyntaxError
  except SyntaxError:

    #the if statement is temporary. Must be removed once checkmate and draw situations have been tested
    if not checkmate or not draw:
      print('SyntaxError in lines 78-115')
    break

  #resetting all the variables to their initial values (because they got altered when checking for draw/checkmate/check)
  piece_position = original_values['piece_position'].copy()
  original_piece_position = original_values['original_piece_position'].copy()
  selected_coin = original_values['selected_coin']
  colour = original_values['colour']

  try:
    if not check:
      print(piece_range_display(colour))
    else: #written with respect to training the bot section of code
      if notation[0] == "K":
        print(piece_range_display(colour))
      else:
        board[piece_position_list[turn]["king"][0][0]][piece_position_list[turn]["king"][0][1]] = 15
        piece_position = piece_position_list[turn]["king"][0]
        original_piece_position = piece_position.copy()
        selected_coin = "15"
        check_counter = 0
        allowed_positions = []

        for horse_range in piece_range_list[opp]["horse"]:
          if piece_position_list[turn]["king"] in horse_range:
            if notation[0] != "K":
              check_counter = 2
              break
        for attacks in piece_range_display(colour)[1]:
          if check_counter > 1:
            if notation[0] != "K":
              print("Invalid move. King is under check!")
              allowed_positions = []
              break
          elif str(board[attacks[0]][attacks[1]])[0] == opp +1 and str(board[attacks[0]][attacks[1]])[1] in ["2","4","5"]:
            check_counter+=1
            for a in piece_position_list[opp]:
              if a in ["rook","bishop","queen"]:
                if attacks in piece_position_list[opp][a]:
                  allowed_positions += piece_range_list[opp][a][piece_position_list[opp][a].index(attacks)].intersection(piece_range_display(colour)[1])

        if moving_piece not in allowed_positions and notation[0] != "K":
          print("Invalid move. King is under check!")

        piece_position = original_values['piece_position'].copy()
        original_piece_position = original_values['original_piece_position'].copy()
        selected_coin = original_values['selected_coin']
        colour = original_values['colour']






    while True: #statement can be written outside of existing try-except loop
      moving_piece= input("Choose a square to move to in the format 'yx' \n y = y-coordinate\n x = x-coordinate")
      try:
        if [int(moving_piece[0]),int(moving_piece[1])] not in piece_range_display(colour)[1]:  #if the piece cannot move to the inputted square coordinate then an error is raised which is handled accordingly
            raise IndexError
        else:
          #move is registered and it's the opponent's chance to play
          move_history.append([piece_position,list(map(int,list(moving_piece))),selected_coin])
                                              #move_history stored values in the format [[initial position],[final position]]
          #EN PASSANT (TESTED)
          if [int(moving_piece[0]),int(moving_piece[1])] == en_passant:
            board[double_moved_pawn[0]][double_moved_pawn[1]] = 0

          #Setting the new positions
          board[int(moving_piece[0])][int(moving_piece[1])] = selected_coin             #the selected piece is now moved to the new position
          board[piece_position[0]][piece_position[1]] = 0            #old position of piece is vacated (by turning the value of the position to 0)

        #PAWN PROMOTION (TESTED)
          if int(moving_piece[0]) in [7,0] and selected_coin[1] == '1':
            while True:
              promotion = input("\n What do you want to promote your pawn to? \n 1) Queen \n 2) Horse \n 3) Rook \n 4) Bishop \n")
              if input_text_filter(promotion) in ["queen","horse","rook","bishop"]:
                selected_coin = selected_coin[0] + str(piece_to_number[input_text_filter(promotion)])
                board[int(moving_piece[0])][int(moving_piece[1])] = selected_coin
                break
              else:
                print("\n You have to promote the pawn! Choose a valid piece.")

        #CASTLING (TESTED)
          castle_side = int(moving_piece[1]) - piece_position[1]
          if selected_coin[1] == '6' and abs(castle_side) == 2 :

            #king castles to the left (from whites perspective. i.e, when white is at the bottom of the board)
            if castle_side == -2:
              board[piece_position[0]][piece_position[1]-1] = board[piece_position[0]][piece_position[1]-4]
              board[piece_position[0]][piece_position[1]-4] = 0

            #king castles to the right (from whites perspective. i.e, when white is at the bottom of the board)
            elif castle_side == 2:
              board[piece_position[0]][piece_position[1]+1] = board[piece_position[0]][piece_position[1]+3]
              board[piece_position[0]][piece_position[1]+3] = 0


          #the move is registered
          original_board = board.copy()

          #Change of turns
          if turn == 'black':
            turn = 'white'
          else:
            turn = 'black'
          break
      except KeyboardInterrupt:
        break
      except:
        print('Enter a valid position!')
  except KeyboardInterrupt:
    break
  except:
    print('Enter a valid square! \n')

Choose your side : 1) Black 2) Whiteblack
[[ 0.  0. 26. 22.  0.  0.  0. 22.]
 [21.  0. 11.  0.  0. 15. 21. 21.]
 [ 0.  0.  0.  0.  0.  0. 24.  0.]
 [ 0. 14. 11. 21. 23.  0.  0.  0.]
 [ 0. 15.  0.  0. 11.  0.  0.  0.]
 [11.  0. 14.  0.  0.  0.  0.  0.]
 [14. 11.  0.  0.  0. 11.  0. 11.]
 [12.  0.  0.  0. 16.  0.  0. 12.]]
Choose a piece's square in the format 'yx' or resign 
 y = y-coordinate
 x = x-coordinate34
{'piece_position': [3, 4], 'original_piece_position': [3, 4], 'selected_coin': '23', 'colour': '2'}
[array([[ 0.,  0., 26., 22.,  0.,  0.,  0., 22.],
       [21.,  0., 11., 99.,  0., 99., 21., 21.],
       [ 0.,  0., 99.,  0.,  0.,  0., 24.,  0.],
       [ 0., 14., 11., 21., 23.,  0.,  0.,  0.],
       [ 0., 15., 99.,  0., 11.,  0., 99.,  0.],
       [11.,  0., 14., 99.,  0., 99.,  0.,  0.],
       [14., 11.,  0.,  0.,  0., 11.,  0., 11.],
       [12.,  0.,  0.,  0., 16.,  0.,  0., 12.]]), [[1, 5], [5, 3], [4, 6], [2, 2], [4, 2], [5, 5], [1, 3]]]
Choose a square to move to in th

KeyboardInterrupt: 

In [ ]:
a = {"a":30}
for b in a:
  print(b)

a


In [ ]:
import numpy as np

#creating the board
board = np.zeros((8,8))

board[0] = np.array([0, 26, 0, 0, 0, 0, 0, 22])
board[1] = np.array([21, 11, 0, 0, 0, 21, 21, 21])
board[2] = np.array([0, 0, 0, 21, 0, 0, 24, 0])
board[3] = np.array([0, 14, 13, 0, 23, 0, 0, 0])
board[4] = np.array([0, 15, 0, 0,11, 0, 0, 0])
board[5] = np.array([11, 0, 14, 0, 0, 0, 0, 0])
board[6] = np.array([16, 11, 0, 0, 0, 11, 0, 11])
board[7] = np.array([0, 0, 0, 0, 0, 0, 0, 25])

original_board = board.copy()
print(board)

[[ 0. 26.  0.  0.  0.  0.  0. 22.]
 [21. 11.  0.  0.  0. 21. 21. 21.]
 [ 0.  0.  0. 21.  0.  0. 24.  0.]
 [ 0. 14. 13.  0. 23.  0.  0.  0.]
 [ 0. 15.  0.  0. 11.  0.  0.  0.]
 [11.  0. 14.  0.  0.  0.  0.  0.]
 [16. 11.  0.  0.  0. 11.  0. 11.]
 [ 0.  0.  0.  0.  0.  0.  0. 25.]]


In [ ]:
#NEXT STEP: Convert standard chess notations to bot readable notations

#if any of these variables become 'True', the game ends

piece_positions = {"pawn":[],"bishop":[],"rook":[],"king":[],"horse":[],"queen":[]} #should store the positions of all the pieces
piece_ranges = {"pawn":[],"bishop":[],"rook":[],"king":[],"horse":[],"queen":[]}
number_to_piece = {1:"pawn",2:"rook",3:"horse",4:"bishop",5:"queen",6:"king"}
checkmate = False
draw= False
pieces = []
for a in len(range(board)):
  for b in len(range(board[a])):
    if str(board[a][b])[0] == "1":
      pieces.append(str(a)+str(b))
    piece_positions[number_to_piece[int(str(board[a][b])[1])]].append(str(a)+str(b))



#working of the game
for a in pieces:
  opposition_range = []

  #variables for en passant
  en_passant = []
  double_moved_pawn = []
  print(original_board)
  selected_square = list(a)
  piece_position = list(map(int,selected_square))       #piece_position = position of the piece selected by the playe

  original_piece_position = piece_position[:] #initial position of the selected piece
  selected_coin = str(int(board[int(piece_position[0])][int(piece_position[1])])) #selected coin = piece that is selected
  colour = selected_coin[0]

  #stores all info about a certain selected piece
  original_values = {'piece_position': piece_position,
                'original_piece_position':original_piece_position,
                'selected_coin': selected_coin,
                'colour': colour}
  if piece_range_display(colour)[1] == []:
    print("\n This piece cannot move anywhere. Choose a valid piece position \n")
    continue

  piece_ranges[number_to_piece[int(selected_coin[1])]].append(piece_range_display(colour)[1])

In [ ]:
import numpy as np

#creating the board
board = np.zeros((8,8))

board[0] = np.array([0, 26, 0, 0, 0, 0, 0, 22])
board[1] = np.array([21, 11, 0, 0, 0, 21, 21, 21])
board[2] = np.array([0, 0, 0, 21, 0, 0, 24, 0])
board[3] = np.array([0, 14, 13, 0, 23, 0, 0, 0])
board[4] = np.array([0, 15, 0, 0,11, 0, 0, 0])
board[5] = np.array([11, 0, 14, 0, 0, 0, 0, 0])
board[6] = np.array([16, 11, 0, 0, 0, 11, 0, 11])
board[7] = np.array([0, 0, 0, 0, 0, 0, 0, 25])

original_board = board.copy()
print(board)

moving_piece = ""

position_conversion = {"a":0,"b":1,"c":2,"d":3,"e":4,"f":5,"g":6,"h":7} #for converting position letters in the notation to according index numbers
piece_positions = {"pawn":[],"bishop":[],"rook":[],"king":[],"horse":[],"queen":[]} #should store the positions of all the pieces
piece_ranges = {"pawn":[],"bishop":[],"rook":[],"king":[],"horse":[],"queen":[]} #should store the range of each piece for all pieces

first_digit= {'white': '1','black':'2'}
first_digit_2 = {1:'white',2:'black'}
second_digit = {1:'pawn',2:'rook',3: "horse",4: 'bishop',5: 'queen',6:'king'}
piece_to_number = {'pawn':1,'rook':2,"horse":3, 'bishop':4, 'queen':5,'king':6}
notation_to_piece = {"R":"rook","N": "horse", "B": "bishop","Q": "queen","K":"king"}

notation = input('')

colour = "white" #first move is always of white

#for en passant
if colour == "black":
  opposite_increment = -1
else:
  opposite_increment = 1

#checking for castling notations
if notation == "O-O-O" or notation == "0-0-0":
  castle_side = 2 #long hand castling
  print("long hand castling")
  print("New ranges for king and rook") #check which rook had long hand castling in its range and change its range accordingly
elif notation == "O-O" or notation == "0-0":
  castle_side = -2 #short hand castling
  print("short hand castling")
    print("New ranges for king and rook") #check which rook had short hand castling in its range and change its range accordingly


#if a piece is captured then the position to which it moves is defined after the x in the notation
if "x" in notation:

  moving_piece = notation[notation.index("x")+1:notation.index("x")+3]
  moving_piece = str(position_conversion[moving_piece[0]]) +str(8 - int(moving_piece[1])) #converts standard notation to bot readable form
                                                          #we do str(int(moving_piece[1])-1 as our column index goes 0,1,2,3,4 but in standard notation it goes 1,2,3,4,5
  print("moving_piece = "+ moving_piece)

#pawn notation conversion
if notation[0] not in ['R,N,B,Q,K']:
  selected_coin = colour +'1'
  if "x" in notation:
    print("pawn capture = True")
    #first letter helps us find the position of the pawn. characters after "x" help us find the position to which the pawn moves
    print("Removed captured piece from piece_positions and piece_ranges")

  #standard pawn notation case
  else:
    moving_piece = notation[0:2] #moving_piece is the position to which the selected piece moves
    moving_piece = str(position_conversion[moving_piece[0]]) +str(8 - int(moving_piece[1])) #converts standard notation to bot readable form
    print("moving_piece = "+ moving_piece)

    #checking which pawn can move to the position mentioned in the standard notation
    print("range_check")
    print("move made") #the corresponding index of the pawn is used to change the range of the pawn to its new range in piece_ranges

  #for pawn promotion
  if "=" in notation:
    promotion = notation_to_piece[notation[notation.index("=")+1]]
    print("Removed captured piece from piece_positions and piece_ranges") #remove the piece moved
    print("Added new range to piece_ranges and new position to piece_position") #index of new range in piece_ranges = index of new position in piece_position

#to directly identify check while training
if notation[-1] == '+':
  check = True
  notation = notation[0:-1]
  print("Check!")

#to directly identify checkmate while training
elif notation[-1] == "#":
  checkmate = True
  notation = notation[0:-1]
  print("Checkmate!")

#notation conversion for all other pieces
if notation[0] in ['R,N,B,Q,K']:
  if "x" in notation:
    notation = notation[0:notation.index("x")] + notation[notation.index("x")+1:] #x does not make a difference whether it exists or not in the notation.
                                        #The bot works in such a way that if a piece moves to a square containing another piece, the other piece is automatically captured
    print("Removed captured piece from piece_positions and piece_ranges") #obtain two letters after x. Check which index in piece positions has the same position as the two letters after x.
                                                                          #Remove the piece. Use the same index to remove the piece from piece_ranges
  selected_coin = colour + str(piece_to_number[notation_to_piece[notation[0]]]) #1st letter of notation indicates the piece
  notation = notation[1:]

  moving_piece = notation[-2:]
  moving_piece = str(position_conversion[moving_piece[0]]) +str(int(8 - moving_piece[1]))

  notation = notation[0:-2]

  #if there are two of the same pieces that can move to the same square
  if len(notation) == 1:
    for a in len(range(piece_positions[second_digit[int(selected_coin[1])]])):
      if notation in piece_positions[second_digit[int(selected_coin[1])]][a] or str(position_conversion[notation]) in piece_positions[second_digit[int(selected_coin[1])]][a]:
        piece_position = [int(piece_positions[second_digit[int(selected_coin[1])]][0]), int(piece_positions[second_digit[int(selected_coin[1])]][1])]

  #if there are three of the same pieces that can move to the same square such that two of them are located in the same row and the other two are located in the same column (very rare)
  elif len(notation) == 2:
    piece_position = [position_conversion[notation[0]], 8 - int(notation[1])]


['lmao', 'lol']